In [4]:
from random import randint, uniform
from util import generate_problem, verify_witness
from matplotlib import pyplot as plt

In [132]:
def random_gene(T, r):
    num_variables = max([max(t['i'], t['j']) for t in T])
    return [0] + [randint(-r, r) for _ in range(num_variables)]

def sweep_line(intervals):
    L = []
    for l, r in intervals:
        L.append( (l, 1) )
        L.append( (r, -1) )
        
    L.sort(key=lambda a: 10*a[0] - a[1])
    
    c = 0
    top = -1
    top_index = 0
    
    for index, (point, increment) in enumerate(L):
        c += increment
        if c > top:
            top = c
            top_index = index
    
    sol = int( (L[top_index][0] + L[top_index+1][0])/2 )
    
    # returns the number of constraints unsatisfied and time point
    return len(intervals) - top, sol

def walk_gene(gene, T, pick_best_gene=True):
    num_variables = max([max(t['i'], t['j']) for t in T])
    
    constraints_per_variable = [list() for i in range(num_variables + 1)]
    failed_per_variable = [0 for i in range(num_variables + 1)]
    # at index i should contain availability intervals (given other variables are fixed.)
    # remark: existence of an interval is local with respect to one constraint
    # remark: so if intersection is empty, then there is no "safe" place to put this variable
    
    for c in T:
        i, j = c['i'], c['j']
        intervals = c['intervals']
        for l, r in intervals:
            # X_j - X_i in (l, r)
            # fix X_j to genes[j]
            # so genes[j] - X_i in (l, r)
            # then X_i in [X_j - r, X_j - l]
            constraints_per_variable[i].append( (gene[j] - r, gene[j] - l) )
            
            # now for j:
            constraints_per_variable[j].append( (gene[i] + l, gene[i] + r) )
            
            
            if gene[j] - gene[i] < l or gene[j] - gene[i] > r:
                failed_per_variable[i] += 1
                failed_per_variable[j] += 1
            
    
    # for each constraint, sweep line, find best option
    # (later? we want to pick whatever REDUCES the unsatisfieds the most)
    # for now: pick a random variable and modify it according to sweep_line
    # if no later, then we can make the loop upstairs simpler (to only do stuff if "i" is mentioned)

    
    if pick_best_gene:
        reduction = -1
        best_gene = -1
        new_gene_base = 0
        for i in range(1, num_variables+1):
            faileds = failed_per_variable[i]
            new_faileds, new_gene_base = sweep_line(constraints_per_variable[i])
            
            if faileds - new_faileds >= reduction: #important to have \geq so it doesnt get stuck on 1 if 1 is best
                reduction = faileds - new_faileds
                best_gene = i
                best_gene_base = new_gene_base
        gene[best_gene] = best_gene_base
    else:
        i = randint(1, num_variables)
        gene[i] = sweep_line(constraints_per_variable[i])[1]
        

    return gene
    
def walk_tcsp(T, r, max_iterations, max_flips, pick_best_gene):
    best_gene = None
    best_gene_failed = None
    
    for i in range(max_iterations):
        gene = random_gene(T, r)
        
        for j in range(max_flips):
            gene_failed = verify_witness(gene, T)
            
            if not best_gene or len(gene_failed) < len(best_gene_failed):
                best_gene = gene
                best_gene_failed = gene_failed
            
            if len(gene_failed) == 0:
                break
                
            gene = walk_gene(gene, T, pick_best_gene)
                
        # necessary for double break after flips loop
        if len(gene_failed) == 0: break
    
    print(f'num iterations: {i+1}')
    print('best gene:', best_gene)
    print('constraints failed:', len(best_gene_failed), 'out of:', len(T)) 
    print('failed constraints:', best_gene_failed)

In [133]:
T = generate_problem(
    50,
    min_intervals=1,
    max_intervals=1,
    scaling_factor=1
)

In [73]:
walk_tcsp(T, 100, 50, 50, pick_best_gene=False)

num iterations: 50
best gene: [0, -26, -4, -26, -42, -66, 18, -23, 11, -6, -37, 48, -48, -36, 48, -14, 31, -5, 82, -73, 98, -29, -20, 0, -18, 12, -6, 11, 1, -25, -54, 16, 20, -9, -39, 15, -6, 0, 45, -33, 66, -1, -3, 15, -21, 80, -13, -12, -18, -40]
constraints failed: 117 out of: 339
failed constraints: [{'i': 1, 'j': 20, 'intervals': [(-70, -49)]}, {'i': 1, 'j': 25, 'intervals': [(-51, -37)]}, {'i': 1, 'j': 28, 'intervals': [(-91, -21)]}, {'i': 1, 'j': 44, 'intervals': [(64, 70)]}, {'i': 2, 'j': 13, 'intervals': [(8, 59)]}, {'i': 2, 'j': 19, 'intervals': [(-33, 74)]}, {'i': 2, 'j': 21, 'intervals': [(0, 4)]}, {'i': 2, 'j': 22, 'intervals': [(46, 100)]}, {'i': 2, 'j': 25, 'intervals': [(-23, -23)]}, {'i': 3, 'j': 16, 'intervals': [(-18, -3)]}, {'i': 3, 'j': 17, 'intervals': [(-97, -59)]}, {'i': 3, 'j': 18, 'intervals': [(-86, 6)]}, {'i': 3, 'j': 20, 'intervals': [(64, 94)]}, {'i': 3, 'j': 28, 'intervals': [(36, 98)]}, {'i': 3, 'j': 30, 'intervals': [(14, 97)]}, {'i': 3, 'j': 37, 'inter

In [4]:
l = [(0, 10), (1, 5), (1, 6), (6, 10)]

In [5]:
sweep_line(l)

(1, 3)

In [94]:
# problem sets:
SIMPLE = {
    'constraint_probability': 0.25,
    'min_intervals': 1,
    'max_intervals': 1,
    'scaling_factor': 1,
}

BINARY = {
    'constraint_probability': 0.25,
    'min_intervals': 2,
    'max_intervals': 2,
    'scaling_factor': 1,
}

FIVE_MAX = {
    'constraint_probability': 0.25,
    'min_intervals': 1,
    'max_intervals': 5,
    'scaling_factor': 1,
}

## Graphing effectiveness of walking a gene

In [96]:
def test(T, r, max_flips):
    unsats = []
    gene = random_gene(T, r)

    for j in range(max_flips):
        gene_failed = verify_witness(gene, T)
        unsats += [len(gene_failed)]

        if len(gene_failed) == 0:
            break

        gene = walk_gene(gene, T)

    return unsats

In [97]:
runs = 50
run_results = []

for _ in range(runs):
    T = generate_problem(
        variables=20,
        **BINARY,
    )
    
    arr = test(T, r=100, max_flips=100)
    
    starting = arr[0]
    ending = arr[-1]
    decrease = (arr[0]-arr[-1]) / arr[0]
    
    # TODO, problem: we do not know if it is even solvable!
    # will have to confirm with backtracking, or make sure to generate only solvable problems
    solved = arr[-1] == 0
    
    print(starting, ending, f'{decrease:.2f}', solved)

25 7 0.72 False
33 5 0.85 False
26 3 0.88 False
21 3 0.86 False
33 8 0.76 False
28 6 0.79 False
28 5 0.82 False
30 8 0.73 False
33 8 0.76 False
24 5 0.79 False
34 7 0.79 False
31 5 0.84 False
31 5 0.84 False
26 1 0.96 False
39 11 0.72 False
26 5 0.81 False
32 5 0.84 False
28 4 0.86 False
15 6 0.60 False
29 3 0.90 False
23 2 0.91 False
28 5 0.82 False
25 6 0.76 False
41 12 0.71 False
23 6 0.74 False
32 8 0.75 False
36 6 0.83 False
34 7 0.79 False
28 4 0.86 False
27 4 0.85 False
27 8 0.70 False
27 2 0.93 False
19 5 0.74 False
27 4 0.85 False
28 5 0.82 False
30 5 0.83 False
30 4 0.87 False
31 4 0.87 False
27 3 0.89 False
24 2 0.92 False
40 4 0.90 False
29 3 0.90 False
23 2 0.91 False
30 3 0.90 False
24 2 0.92 False
30 3 0.90 False
30 5 0.83 False
33 11 0.67 False
28 8 0.71 False
33 9 0.73 False


In [18]:
# Note:
# if returned length is less, that means the heuristic solved the problem
# check whether it ever increases.

## Graphing pick_best_gene vs random gene selection for walking

In [137]:
runs = 50
run_results = []

count_same = 0
count_better = 0
count_worse = 0
reductions = []

for _ in range(runs):
    T = generate_problem(
        variables=50,
        **BINARY,
    )
    
    gene = random_gene(T, 100)
    best_gene = [i for i in gene]
    rand_gene = [i for i in gene]
    
    pick_best = len(verify_witness(
        walk_gene(best_gene, T, pick_best_gene=True),
        T)
    )
    pick_random = len(verify_witness(
        walk_gene(rand_gene, T, pick_best_gene=False),
        T)
    )
    
    reductions += [pick_random - pick_best]
    
    if pick_best == pick_random:
        count_same += 1
    elif pick_best < pick_random:
        count_better += 1
    else:
        count_worse += 1
    
print('same', count_same)
print('better', count_better)
print('worse', count_worse)

from statistics import stdev
print('avg reduction', sum(reductions)/len(reductions))
print('stddev reduction', stdev(reductions))

same 1
better 49
worse 0
avg reduction 5.24
stddev reduction 2.343553903361129


## and now, what about repeated application of best/random?

as in, how much can it reduce, just like the random gene vs n walks test

In [136]:
runs = 5
walks = 50
run_results = []

count_same = 0
count_better = 0
count_worse = 0

for _ in range(runs):
    T = generate_problem(
        variables=10,
        **BINARY,
    )
    
    gene = random_gene(T, 100)
    best_gene = [i for i in gene]
    rand_gene = [i for i in gene]
    
    for _ in range(runs):
        best_gene = walk_gene(best_gene, T, pick_best_gene=True)
        rand_gene = walk_gene(rand_gene, T, pick_best_gene=False)
    
    pick_best = len(verify_witness(best_gene, T))
    pick_random = len(verify_witness(rand_gene, T))
    
    if pick_best == pick_random:
        count_same += 1
    elif pick_best < pick_random: # \lt because verify_witness is like golf.
        count_better += 1
    else:
        count_worse += 1
    
print('same', count_same)
print('better', count_better)
print('worse', count_worse)

same 1
better 4
worse 0
